## Map/Reduce Semantics and Systems

### Types and transformations
* Map is a transformation
  * Input domain to output domain
* Reduce is a collection
  * No domain change
  
$$
map (k1, v1) \rightarrow list(k2,v2) \\
reduce (k2, list(v2)) \rightarrow list(v2)
$$

* Google C++ implementation is based all on strings
  * User code must convert to structured types
* Hadoop! has type wrappers


